<a href="https://colab.research.google.com/github/rana14ce/Dataset-of-the-Multinational-Pile/blob/v2/GitHub_XGBoost_with_5_fold_cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install optunity

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for optunity: filename=Optunity-1.1.1-py3-none-any.whl size=72027 sha256=c8ad7cab9df3f4dce1a75e3af4f3b2c9bb106c08ee53189f19636b96071afe5b
  Stored in directory: /root/.cache/pip/wheels/83/98/40/b9956c3386a045f035f9edbf894edd04713db92f29fb58a225
Successfully built optunity


In [5]:
# CTRL+Shift+Space bar = docstring
# CTRL+Space bar = autocomplete

# Import necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

import optunity
import optunity.metrics

In [6]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [7]:
url = 'https://raw.githubusercontent.com/rana14ce/Dataset-of-the-Multinational-Pile/main/Dataset%20of%20the%20Multinational%20Pile.csv'
data = pd.read_csv(url)

In [8]:
data.head()

,Phi_s,Phi_t,L_m,A_m2,Sv _KN/m2,Qm_KN
0,33.0,38.0,255,24.5,0.1310,2615
1,34.0,37.5,206,19.8,0.2230,3675
2,33.0,38.0,223,21.5,0.1310,2164
3,33.0,37.5,210,20.2,0.1468,3042
4,33.0,37.0,206,19.9,0.1821,2856


In [9]:
X = data.drop('Qm_KN', axis=1)
y = data['Qm_KN']

In [10]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
# Define the objective function for PSO
def objective_function(max_depth, gamma, learning_rate, n_estimators):
    model = XGBRegressor(max_depth=int(max_depth), gamma=gamma, learning_rate=learning_rate, n_estimators=int(n_estimators))
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
    mean_rmse = np.mean(np.sqrt(-cv_scores))
    return mean_rmse

# Define the hyperparameter search space
search_space = {
    'max_depth': [3, 7],
    'gamma': [0, 0.5],
    'learning_rate': [0.01, 0.2],
    'n_estimators': [100, 300]
}

# Perform PSO
optimal_params, _, _ = optunity.minimize(objective_function, num_evals=100, **search_space)
print("Optimal Parameters: ", optimal_params)

Optimal Parameters:  {'max_depth': 5.42023571009677, 'gamma': 0.42712103164446613, 'learning_rate': 0.016128093816002723, 'n_estimators': 107.88853660648806}


In [13]:
# Train the best model with the optimal hyperparameters
best_model = XGBRegressor(max_depth=int(optimal_params['max_depth']), gamma=optimal_params['gamma'],
                          learning_rate=optimal_params['learning_rate'], n_estimators=int(optimal_params['n_estimators']))
best_model.fit(X_train_scaled, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.42712103164446613, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.016128093816002723,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=107, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [14]:
# Perform 5-Fold Cross-Validation with the best model
cv_scores = cross_val_score(best_model, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
cv_rmse = np.sqrt(-cv_scores)
mean_cv_rmse = np.mean(cv_rmse)

In [15]:
# Predict on the training set
y_train_pred = best_model.predict(X_train_scaled)

# Predict on the test set
y_test_pred = best_model.predict(X_test_scaled)

In [16]:
# Evaluate the model on the training set
train_r2 = r2_score(y_train, y_train_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))

# Evaluate the model on the test set
test_r2 = r2_score(y_test, y_test_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

print("5-Fold CV RMSE: ", cv_rmse)
print("Mean CV RMSE: ", mean_cv_rmse)
print("Training R^2: ", train_r2)
print("Training RMSE: ", train_rmse)
print("Test R^2: ", test_r2)
print("Test RMSE: ", test_rmse)


5-Fold CV RMSE:  [ 895.54552993  524.27614477 1004.03763886  846.89340688 1002.63850449]
Mean CV RMSE:  854.6782449856652
Training R^2:  0.8834880436500692
Training RMSE:  389.8424850260029
Test R^2:  0.18268139092581626
Test RMSE:  1244.8837023683152
